In [1]:

### import all the dependencies 
import requests
import csv
import json
import pandas as pd
import requests, json
import re
import configparser
import warnings
warnings.filterwarnings('ignore')

In [2]:
## Define all variables 
configFile = "config.cfg"
config = configparser.ConfigParser()
config.read(configFile)


['config.cfg']

In [3]:
### Main Program starts here  - Now we will read data form CSV
print("Starting to read from csv , data is huge so it may take few seconds")
main_customers_df = pd.read_csv("../data/Main_customers.csv", low_memory=False)
print('Shape of exports read from CSV : {}'.format(main_customers_df.shape))

Starting to read from csv , data is huge so it may take few seconds
Shape of exports read from CSV : (711157, 35)


In [4]:
##List the names of columns 
customers_df=main_customers_df
customers_df.columns


Index(['Unnamed: 0', 'EventModDead', 'EventModError', 'EventModFrustrated',
       'EventStart', 'EventTargetSelectorTok', 'EventTargetText', 'EventType',
       'IndvId', 'PageActiveDuration', 'PageAgent', 'PageBrowser',
       'PageDevice', 'PageDuration', 'PageId', 'PageIp', 'PageLatLong',
       'PageNumErrors', 'PageNumInfos', 'PageNumWarnings',
       'PageOperatingSystem', 'PageRefererUrl', 'PageUrl', 'SessionId',
       'UserAppKey', 'UserDisplayName', 'UserEmail', 'UserId',
       'user_anonymousId_str', 'user_firstName_str', 'user_lastName_str',
       'user_serviceName_str', 'user_userHash_str', 'user_userId_str',
       'user_username_str'],
      dtype='object')

In [5]:
## Data Cleaning and filtering 
#dropping an Column not needed 
print('number of columns  before :{}'.format(len(customers_df.columns)))
customers_df.drop('Unnamed: 0', axis=1, inplace=True)
print('number of columns  After :{}'.format(len(customers_df.columns)))

number of columns  before :35
number of columns  After :34


In [11]:
## Need to Add a column as Domain 
print('Adding domain column')

customers_df['Domain'] = customers_df['UserEmail'].str.split(pat="@", expand=True)[1]
print('Shape of exports read from CSV : {}'.format(customers_df.shape))

Adding domain column
Shape of exports read from CSV : (711157, 35)


In [12]:

# drop rows that doesn't have an email
#customers_df = customers_df.dropna(subset=['UserEmail'])

# remove all the rows that use internal email or not valid external customers  emails
#Read the info on emails fron config file 
dropemails= config.get('Emails', 'dropEmailList')
dropemailsList = dropemails.split(",")
external_customers_df = customers_df[~ customers_df.Domain.isin(dropemailsList)]

print('Shape of dataframe after dropping internal users : {}'.format(external_customers_df.shape))



Shape of dataframe after dropping internal users : (78291, 35)


In [13]:
#quick preview of data
external_customers_df.head()
# some email address have both lower and upper case and counted as double
external_customers_df['Domain'] = external_customers_df['Domain'].str.lower()



In [14]:
## Dropping unwanted columns 
external_customers_df = external_customers_df[['Domain','UserAppKey','user_serviceName_str','UserId','IndvId','SessionId',
                                       'PageUrl','PageRefererUrl','PageBrowser','PageDevice','PageDuration','PageActiveDuration',
                                        'PageLatLong',
                                       'EventStart','EventType']]
print('Shape of dataframe after dropping extra columns: {}'.format(external_customers_df.shape))                                    

Shape of dataframe after dropping extra columns: (78291, 15)


In [15]:
##Renaming columns
external_customers_df.rename(columns={'UserAppKey':'UserEmail','user_serviceName_str':'serviceName'},inplace = True)
external_customers_df.columns

Index(['Domain', 'UserEmail', 'serviceName', 'UserId', 'IndvId', 'SessionId',
       'PageUrl', 'PageRefererUrl', 'PageBrowser', 'PageDevice',
       'PageDuration', 'PageActiveDuration', 'PageLatLong', 'EventStart',
       'EventType'],
      dtype='object')

In [16]:
## Printing list of unique services we have collected data for 
list(external_customers_df.serviceName.unique())

['Data Logging', 'SDK', 'Data Package']

In [ ]:
#Masking Company Names 
domainList  = list(external_customers_df.Domain.unique())
counter = 1
for domain in domainList  :
    newDomain = "Company" + str(counter)+".com"
    print(newDomain)
    external_customers_df.replace(domain,newDomain,inplace =True,regex=True)
    counter=counter +1  

print("Masking done")


Company1.com
Company2.com
Company3.com
Company4.com
Company5.com
Company6.com
Company7.com
Company8.com
Company9.com
Company10.com
Company11.com
Company12.com
Company13.com
Company14.com
Company15.com
Company16.com


In [17]:
### Writing  masked /filtered and clean data to CSV ,from here always read the local file names external_customers.csv
external_customers_df.to_csv("../data/external_customers.csv")
print("external Customer data  Successfully written to external_customers.csv ")

external Customer data  Successfully written to external_customers.csv 
